<a href="https://colab.research.google.com/github/ArnavMehrotra/ArNet/blob/main/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
%%shell
if [ ! -d ArNet ]; then
  git clone https://github.com/ArnavMehrotra/ArNet
fi

In [8]:
import time
import ctypes
import numpy as np
import os

In [9]:
%%shell
cd ArNet/
git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0 (from 0)
Unpacking objects: 100% (3/3), 263 bytes | 263.00 KiB/s, done.
From https://github.com/ArnavMehrotra/ArNet
   1212b29..9e1aab8  main       -> origin/main
Updating 1212b29..9e1aab8
Fast-forward
 optest.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [10]:
!rm -rf *.so
so_name = f"ops_{int(time.time())}.so"
!nvcc -Xcompiler -fPIC -shared -gencode arch=compute_80,code=sm_80 -o {so_name} ArNet/launch.cu ArNet/kernels.cu
LIB = ctypes.CDLL(f"./{so_name}")

In [11]:
import sys
sys.path.append("./ArNet")

from optest import run_tests

run_tests(LIB)

Total time for numpy: 363.907 ms
Total time for young arn: 372.243 ms
Total time for young arn (optimized): 621.249 ms
input 1: 7373x2778 matrix
input 2: 2778x6439 matrix
output: 7373x6439 matrix
input 1: 6704x8334 matrix
input 2: 6704x8334 matrix
output: 6704x8334 matrix
input: 5495x4850 matrix
input 1: 4193x5548 matrix
input 2: 1x5548 matrix
output: 4193x5548 matrix
All tests passed!


In [12]:
# lr = 0.001
# epochs = 100

# J, D, H, K = 64, 32, 16, 4  # batch size, input dim, hidden dim, output classes

# X = np.random.randn(J, D).astype(np.float32)
# Y = np.random.randint(0, K, size=(J, 1)).astype(np.uint32)

# W1 = np.random.randn(D, H).astype(np.float32) * 0.01
# b1 = np.zeros((H,), dtype=np.float32)

# W2 = np.random.randn(H, K).astype(np.float32) * 0.01
# b2 = np.zeros((K,), dtype=np.float32)
# for i in range(epochs):
#   #forward pass
#   M1 = gemm(X, W1)
#   Z1 = biasAdd(M1, b1)
#   A1 = relu(Z1)
#   M2 = gemm(A1, W2)
#   Z2 = biasAdd(M2, b2)
#   A2 = softmax(Z2)
#   probs = A2[np.arange(J), Y.squeeze()]
#   loss = -np.log(probs + 1e-8).mean()

#   print(loss)

#   #backprop
#   dZ2 = gradient(A2, Y)
#   dW2 = gemm(A1.T, dZ2)
#   db2 = sum(dZ2)
#   dA1 = gemm(dZ2, W2.T)
#   dZ1 = dA1 * (Z1 > 0)
#   dW1 = gemm(X.T, dZ1)
#   db1 = sum(dZ1)

#   #update weights
#   W1 -= lr * dW1
#   b1 -= lr * db1

#   W2 -= lr * dW2
#   b2 -= lr * db2